In [1]:
import pandas as pd
import numpy as np

In [2]:
charterdf = pd.read_pickle('../../nowdata/charters_2015.pkl')

In [3]:
list(charterdf.columns)

['NCESSCH',
 'URL',
 'LAT1516',
 'LON1516',
 'AM',
 'AS',
 'BL',
 'HI',
 'HP',
 'TR',
 'TOTFRL',
 'CHARTER_TEXT',
 'WEBSITE',
 'MEMBER',
 'LEVEL',
 'LOCALE15',
 'LEAID',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AMALM',
 'AMALF',
 '

In [4]:
charterdf.shape

(10965, 702)

In [5]:
df_URLs = pd.read_csv("../../nowdata/backups/charter_URLs_2016.csv")  # Holds URL for each school in 'URL' column
df_URLs = df_URLs[["NCESSCH", "URL"]]

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,41,48,52,81,82,100,130,186,213) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
charter_df = pd.merge(charterdf, df_URLs, how="outer", on="NCESSCH") # Add 'URL' column from df_URLs to charter_df, joining on 'NCESSCH'. how='outer' means keep all keys and rows from both DFs, in this case resulting in a full list of all charter schools and URLs

In [9]:
# Making sure URL column is kept
charter_df['URL_y'].head()

0    http://www.maef.net/our-work/programs/accel/#p...
1                             http://ayaprun.lksd.org/
2               https://www.kgbsd.org/ketchikancharter
3                        http://www.tongassschool.org/
4                         https://aquarian.asdk12.org/
Name: URL_y, dtype: object

In [10]:
df_URLs["NCESSCH"] = df_URLs["NCESSCH"].astype(float)# Convert to float type (just in case)
charter_df["NCESSCH"] = charter_df["NCESSCH"].astype(float) # Trim down to just the variables we want: unique ID and URL


In [12]:
charter_df = charter_df.dropna(subset = ['WEBTEXT']) #dropping NaN values in WEBTEXT -- merging creates NaN values

In [13]:
# Keep rows where NO WEBTEXT (0 pages) was gathered (i.e., scrape failed)
missing1 = charter_df[charter_df["WEBTEXT"].apply(len) == 0] 

# Drop rows where NO WEBTEXT (0 pages) was gathered (i.e., scrape failed)
charter_df = charter_df[charter_df["WEBTEXT"].apply(len) > 0] 

# Keep rows where first scraped page is empty
missing2 = charter_df[charter_df["WEBTEXT"].apply(lambda school: len(school[0][3])) == 0]

# Keep rows where all pages are empty
missing3 = charter_df[charter_df["WEBTEXT"].apply(lambda school: sum([len(page[3]) for page in school])) == 0] 

missingdf = pd.concat([missing1, missing2, missing3])

In [14]:
missingdf.shape

(4944, 703)

## Filtering Data 

Determine rows based on if school is not scrapped yet and still open.

Not scrapped: `WEBTEXT` length == 0  
School opened: look at latest SY_STATUS [`SY_STATUS15`]  
    `open:` SY_STATUS15 == 1,3,4,5 or 8  
    `closed:` SY_STATUS15 == 2,6 or 7

In [60]:
#filtered_WEBTEXT = (charterdf['WEBTEXT'].str.len() < 1) 

#no_urls = charterdf.loc[filtered_WEBTEXT]
#no_urls[['SY_STATUS','WEBTEXT']].head()

In [17]:
filtered_STATUS = (missingdf['SY_STATUS15'] == 1) | (missingdf['SY_STATUS15'] == 3) | (missingdf['SY_STATUS15'] == 4) | (missingdf['SY_STATUS15'] == 5) | (missingdf['SY_STATUS15'] == 8)

#filtered_STATUS
charter_df = missingdf.loc[filtered_STATUS]
charter_df.head()

,NCESSCH,URL_x,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,PROG_VALID_COUNT,PROG_VALID_STR,RIT_VALID_COUNT,RIT_VALID_STR,INQUIRY_RATIO,DISCIPLINE_RATIO,ESS_VALID_RATIO,PROG_VALID_RATIO,RIT_VALID_RATIO,URL_y
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://ayaprun.lksd.org/
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,https://familypartnership.asdk12.org/
12,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.173154,-149.895616,8.0,9.0,1.0,23.0,3.0,48.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,https://rilkeschule.asdk12.org/
16,2.003900e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.554091,-151.254006,11.0,1.0,0.0,1.0,0.0,15.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,http://nome.nosd.schoolaccess.net/~acsa/,64.542453,-165.405470,35.0,0.0,1.0,2.0,0.0,8.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,http://nome.nosd.schoolaccess.net/~acsa/


In [18]:
len(df_URLs)

7400

In [19]:
list(charter_df)

['NCESSCH',
 'URL_x',
 'LAT1516',
 'LON1516',
 'AM',
 'AS',
 'BL',
 'HI',
 'HP',
 'TR',
 'TOTFRL',
 'CHARTER_TEXT',
 'WEBSITE',
 'MEMBER',
 'LEVEL',
 'LOCALE15',
 'LEAID',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AMALM',
 'AMALF',


In [21]:
charter_df = charter_df[["NCESSCH", "URL_y"]] # The only variables we need for web-crawling
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") # Getting rid of duplicate NCESSCH columns (shouldn't be looking over the same school twice)

In [22]:
charter_df.shape

(1050, 2)

In [23]:
charter_df = charter_df.drop_duplicates(subset="NCESSCH", keep="first") 

In [33]:
charter_df.columns = ['NCESSCH', 'URL']
charter_df.head() # View of the data

,NCESSCH,URL
1,2.000010e+10,http://ayaprun.lksd.org/
5,2.001800e+10,https://familypartnership.asdk12.org/
12,2.001800e+10,https://rilkeschule.asdk12.org/
16,2.003900e+10,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,http://nome.nosd.schoolaccess.net/~acsa/


In [24]:
charter_df.shape

(1009, 2)

In [32]:
# Taking a look at what charter_df is & exploring values 
charter_df = charter_df.dropna(subset = ["URL"])
charter_df = charter_df[charter_df["URL"] != '0']

charter_df


,NCESSCH,URL
1,2.000010e+10,http://ayaprun.lksd.org/
5,2.001800e+10,https://familypartnership.asdk12.org/
12,2.001800e+10,https://rilkeschule.asdk12.org/
16,2.003900e+10,http://auroraborealis.blogs.kpbsd.k12.ak.us/
28,2.005700e+10,http://nome.nosd.schoolaccess.net/~acsa/
32,2.006000e+10,http://ekc.k12northstar.org/
143,4.000740e+10,http://kaolaz.org/calendar.html
381,4.002090e+10,http://www.canyonviewprep.org/contact-us
443,4.002410e+10,http://www.pecschools.org/
444,4.002410e+10,http://pecschools.org/


In [27]:
# Convert the dataframe to a CSV.
charter_df.to_csv('../data/charter_urls_2016_unscraped.csv')
charter_df.to_csv('../../scrapy-cluster/kafka-monitor/charter_urls_2016_unscraped.csv')

End result: save the resulting data frame as a CSV named charter_URLs_2016_unscraped.csv in web_scraping/data and in scrapy-cluster/kafka-monitor/

## Comparing original `charter_URLs_2016.csv` to a smaller subset

In [25]:
charter_URLs_2016 = pd.read_csv("../../scrapy-cluster/kafka-monitor/charter_URLs_2016.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,41,48,52,81,82,100,130,186,213) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Testing URLs that were able to be scrapped successfully to compare to the way-back machiine link

In [29]:
charter_URLs_2016[['NCESSCH', 'URL']]

,NCESSCH,URL
0,2.000010e+10,http://ayaprun.lksd.org/
1,2.001500e+10,http://www.tongassschool.org/
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter
3,2.001800e+10,https://paideia.asdk12.org/
4,2.001800e+10,http://anccs.asdk12.org/
5,2.001800e+10,https://rilkeschule.asdk12.org/
6,2.001800e+10,http://highlandacademy.asdk12.org/
7,2.001800e+10,http://www.frontiercs.org/
8,2.001800e+10,http://www.winterberrycharterschool.com/
9,2.001800e+10,https://familypartnership.asdk12.org/


In [26]:
charter_URLs_test = charter_URLs_2016[7:8][['NCESSCH', 'URL']]
charter_URLs_test.loc[8] = ['2.001800e+10', 'https://web.archive.org/web/20160313053059/http://www.frontiercs.org/']

charter_URLs_test.to_csv("../../scrapy-cluster/kafka-monitor/way-back_test.csv")

charter_URLs_test



,NCESSCH,URL
7,2.0018e+10,http://www.frontiercs.org/
8,2.001800e+10,https://web.archive.org/web/20160313053059/htt...


## Testing way-back link compared to regular links to see if any web-scraping output

In [83]:
columns = ['URL']
link = ['https://www.kgbsd.org/ketchikancharter'
        ,'https://web.archive.org/web/20140209165819/https://www.kgbsd.org/ketchikancharter'
       , 'http://aquarian.asdk12.org/events/pictureday.html'
       , 'https://web.archive.org/web/20150921144050/http://aquarian.asdk12.org/events/internetsafetyparentnight.html']
df = pd.DataFrame(link, columns = columns)

df.to_csv("../../scrapy-cluster/kafka-monitor/way-back_test.csv")


In [97]:
df

,URL
0,https://www.kgbsd.org/ketchikancharter
1,https://web.archive.org/web/20140209165819/htt...
2,http://aquarian.asdk12.org/events/pictureday.html
3,https://web.archive.org/web/20150921144050/htt...
